# 🍽️ YOLO Food Detector + Resep Makanan

Notebook ini memungkinkan Anda mengunggah gambar makanan, mendeteksi jenis makanan menggunakan model YOLO, dan menampilkan resep terkait dari TheMealDB API.

In [ ]:
# Install dependencies (jalankan sekali jika belum terinstall)
!pip install ultralytics streamlit requests pillow ipywidgets

In [ ]:
# Import library yang diperlukan
from ultralytics import YOLO
import requests
from PIL import Image
from io import BytesIO
import ipywidgets as widgets
from IPython.display import display, Markdown, HTML
import tempfile
import os

In [ ]:
# Label mapping untuk pencarian resep
label_to_search_query = {
    '-': '',
    'beef carpaccio': 'carpaccio',
    'beet salad': 'beet',
    'breakfast burrito': 'burrito',
    'caesar salad': 'caesar salad',
    'caprese salad': 'caprese',
    'chicken quesadilla': 'quesadilla',
    'chicken wings': 'chicken',
    'clam chowder': 'chowder',
    'club sandwich': 'sandwich',
    'deviled eggs': 'egg',
    'dumplings': 'dumplings',
    'eggplant': 'eggplant',
    'falafel': 'falafel',
    'french fries': 'fries',
    'french toast': 'toast',
    'fried rice': 'fried rice',
    'gnocchi': 'gnocchi',
    'greek salad': 'salad',
    'guacamole': 'guacamole',
    'gyoza': 'dumplings',
    'hamburger': 'burger',
    'hot dog': 'hotdog',
    'hummus': 'hummus',
    'ice cream': 'ice cream',
    'lentil soup': 'lentil soup',
    'macaroni and cheese': 'mac and cheese',
    'molokhia': 'soup',
    'mussels': 'mussels',
    'nachos': 'nachos',
    'omelette': 'omelette',
    'onion rings': 'onion',
    'pancakes': 'pancakes',
    'samosa': 'samosa',
    'sashimi': 'sashimi',
    'spring rolls': 'rolls',
    'steak': 'steak',
    'stuffed grape leaves': 'stuffed',
    'sushi': 'sushi',
    'sweet potato': 'sweet potato',
    'tacos': 'tacos',
    'waffles': 'waffles'
}

In [ ]:
# Fungsi untuk YOLO detection
def detect_food_objects(image_path):
    model = YOLO('runs/myfood2/weights/best.pt')  # Ganti dengan path model Anda
    results = model(image_path)
    detections = []
    for r in results:
        for box in r.boxes:
            cls = int(box.cls[0])
            label = model.names[cls]
            detections.append(label)
    return list(set(detections))

In [ ]:
# Fungsi untuk mengambil resep dari TheMealDB
def search_meal_by_name(query):
    url = f'https://www.themealdb.com/api/json/v1/1/search.php?s={query}'
    response = requests.get(url)
    if response.status_code == 200:
        return response.json().get('meals')
    return None

def get_ingredients(meal):
    ingredients = []
    for i in range(1, 21):
        ing = meal.get(f'strIngredient{i}')
        meas = meal.get(f'strMeasure{i}')
        if ing and ing.strip():
            ingredients.append(f'🔸 {ing.strip()} - {meas.strip() if meas else ""}')
    return ingredients

In [ ]:
# Upload gambar di Colab
from google.colab import files
uploaded = files.upload()
image_path = list(uploaded.keys())[0]

In [ ]:
# Proses gambar untuk Colab
image = Image.open(image_path)
display(Markdown('### 📷 Gambar yang Diunggah'))
display(image)
with tempfile.NamedTemporaryFile(delete=False, suffix='.jpg') as temp_file:
    image.save(temp_file, format='JPEG')
    temp_image_path = temp_file.name
labels = detect_food_objects(temp_image_path)
display(Markdown('### 🎯 Hasil Deteksi:'))
for label in labels:
    display(Markdown(f'- **{label}**'))
    query = label_to_search_query.get(label.lower(), label)
    display(Markdown(f"**Resep berkaitan dengan: '{label}'**"))
    meals = search_meal_by_name(query)
    if meals:
        for meal in meals:
            display(Markdown('---'))
            display(Markdown(f'### {meal["strMeal"]}'))
            img_response = requests.get(meal['strMealThumb'])
            img = Image.open(BytesIO(img_response.content))
            display(img)
            display(Markdown(f'**Kategori:** {meal["strCategory"]}  |  **Asal:** {meal["strArea"]}'))
            display(Markdown('**📋 Bahan-bahan:**'))
            for item in get_ingredients(meal):
                display(Markdown(item))
            display(Markdown('**📖 Instruksi:**'))
            display(Markdown(meal['strInstructions']))
            if meal['strYoutube']:
                display(HTML(f'<a href="{meal["strYoutube"]}" target="_blank">▶️ Tonton Video</a>'))
    else:
        display(Markdown(f'❗ Resep untuk "{label}" tidak ditemukan.'))
os.unlink(temp_image_path)

## Petunjuk Penggunaan
1. Jalankan semua sel di atas secara berurutan.
2. Gunakan widget unggah untuk memilih gambar makanan (format .jpg, .jpeg, .png, atau .webp).
3. Setelah mengunggah, notebook akan:
   - Menampilkan gambar yang diunggah.
   - Mendeteksi makanan menggunakan model YOLO.
   - Menampilkan resep terkait dari TheMealDB, termasuk bahan, instruksi, dan tautan video (jika ada).

**Catatan:** Pastikan model YOLO (`runs/myfood2/weights/best.pt`) tersedia di direktori yang benar.